# Deployment

This Jupyter notebook demonstrates the end-to-end deployment of the ct_detector package for elephant camera trap data processing. It covers the setup of data paths, the generation of mosaics, the initialization of trackers, the configuration of callbacks, and the running of predictions with a custom ensemble model. The notebook culminates in post-processing, which includes saving results to an SQLite database and generating mosaic images.

In this cell, we define the folder paths where the camera trap images are stored, along with the localities and camera IDs. These are used to structure and access the relevant images for processing. You can modify the localities, cam_ids, and folders to match your dataset structure.

In [ ]:
import os
from ct_detector.model import DATASETS_DIR

folder_path = os.path.join(DATASETS_DIR, "3", "test", "images")
localities = ["Dibo", "Golom", "Messok"]
dibo_cams = ["dgor1", "dgor2", "dgor3", "dgor4"]
golom_cams = ["gele1", "gele2", "gele3"]
messok_cams = ["mgor2", "mgor3"]
cam_ids = [dibo_cams, golom_cams, messok_cams]
resources = {k: v for k, v in zip(localities, [{cam_id: {} for cam_id in loc_ids} for loc_ids in cam_ids])}
folders = {k: v for k, v in zip(localities, cam_ids)}
print(folders)

This cell defines a function that processes each camera trap subfolder to resize images and create mosaics. The mosaics are helpful for visualizing the results and analyzing the images in bulk. You can adjust the output size and maximum mosaic size multiplier as needed.

In [ ]:
from ct_detector.utils.mosaics import create_mosaics_from_folders


def postprocess_mosaics(folders: dict):
    """
    Process each subfolder in the input folder, resize images, and create mosaics.
    """
    for locality, cam_ids in folders.items():
        for cam_id in cam_ids:
            subdir = os.path.join('./results', locality, cam_id, 'individual_crops')
            create_mosaics_from_folders(subdir, output_size=320, max_size_multiplier=5)

Here, we initialize a tracker for each camera trap dataset and set up a callback for tracking the results. Additionally, metadata (such as locality and camera ID) is added for each camera trap. You can customize this setup to track additional features or modify the callback behavior.

In [ ]:
from ct_detector.model.track import CtTracker
from ct_detector.callbacks.track import track_results
from ct_detector.callbacks.database import postprocess_sqlite_data

for locality, cam_ids in resources.items():
    for cam_id, data in cam_ids.items():
        data['tracker'] = CtTracker()  # Initialize the tracker
        data['track_callback'] = track_results(data['tracker'], persist=True)  # Initialize the tracking callback
        data['metadata'] = {"location": locality, "camera_id": cam_id}  # Add metadata

postprocess_callback = postprocess_sqlite_data(db_path="elephants.db", source_table="image_data")

In this cell, we define the function setup_callbacks, which configures the necessary callbacks for logging prediction data into an SQLite database, saving class mosaics, and displaying results on the screen. You can add more callbacks depending on your needs or modify the existing ones.

In [ ]:
# We will import all the callbacks we need.
from ct_detector.callbacks.database import log_prediction_data_into_sqlite, CLASS_MAP, postprocess_sqlite_data
from ct_detector.callbacks.images import save_cls_mosaic
from ct_detector.callbacks.base import display_results


def setup_callbacks(db_path: str, metadata: dict, output_dir: str, class_map: dict = CLASS_MAP):

    callbacks = {}

    # We set up the callbacks that require it.
    callbacks['db'] = log_prediction_data_into_sqlite(db_path=db_path, table_name="image_data", class_map=class_map, metadata=metadata)
    callbacks['img'] = save_cls_mosaic(output_dir=output_dir, mosaic_size=640, padding=10, class_map=class_map)
    callbacks['etc'] = display_results  # Display results on the screen

    return callbacks

This cell defines the function run_predictions, which runs predictions using a model ensemble. The CtEnsembler is set up with paths to the YOLO models and various predictor options. It also specifies the settings for result merging and applies the configured callbacks during prediction. You can modify the model paths, confidence thresholds, and NMS settings to fit your use case.

In [ ]:
from ct_detector.model.ensemble import CtEnsembler
from ct_detector.model import MODELS, DATASETS, DATASETS_DIR
import os


def run_predictions(data_path: str, callbacks):

    # Create the ensemble
    ensembler = CtEnsembler(
        model_paths=[MODELS['eie_t_1_yolov8m'], MODELS['eie_t_1_yolov9m']], # Specify a list of models
        predictor_overrides={"conf": 0.3, "save_txt": False, "verbose": False, 'imgsz': 640},  # conf, imagsz - for auto-resizing
    )

    # Run predictions on a .txt of images with custom callbacks
    gen = ensembler.predict(
        source=data_path,
        nms_iou_thres=0.5, # threshold for overlap during result merging
        nms_conf_thres=0.3, # threshold of conf considered during result merging
        class_agnostic=False, # whether to consider boxes of any class for merging in case of sufficient overlap or whether only consider same class
        class_merge_map={0:0, 1:1, 2:0, 3:2},  # Merge classes 0 and 2. Specify groups of classes that are together considered for merging.
        _callbacks=callbacks # A list of callback to execute after a result is merged.
    )

    # For each merged frame, do more logic if you want
    for idx, merged_frame in enumerate(gen):
        pass

This final cell loops through all camera trap data, runs predictions using the previously set up callbacks, and finally calls the post-processing functions for both the database and mosaics. You can customize the data path and tweak the post-processing functions if your results require further refinement.

In [ ]:
# For each camera_id in each locality run the predictions, when all is done postprocess the database and mosaics.
data_path = os.path.join(DATASETS_DIR, "2", "test", "images") # or your dataset .yaml or folder of images

for locality, cam_ids in folders.items():
        for cam_id in cam_ids:
            # Put together the source data path
            data_path = os.path.join(folder_path, locality, cam_id)

            # Set up the callbacks
            callbacks_db = setup_callbacks(db_path="elephants.db",
                                           metadata=resources[locality][cam_id]['metadata'],
                                           output_dir=os.path.join('./results', locality, cam_id)
            )
            callbacks = [
                resources[locality][cam_id]['track_callback'],
                callbacks_db['db'],
                callbacks_db['img'],
                callbacks_db['etc']
            ]

            # Run the predictions
            run_predictions(data_path, callbacks)

# Postprocess the database and mosaics
postprocess_callback()
postprocess_mosaics(folders)